# Colab-mmclassification

Original repo: [open-mmlab/mmclassification](https://github.com/open-mmlab/mmclassification)

My fork: [styler00dollar/Colab-mmclassification](https://github.com/styler00dollar/Colab-mmclassification)

In [ ]:
!nvidia-smi

In [ ]:
#@title install
!git clone https://github.com/open-mmlab/mmclassification.git
%cd mmclassification
!pip install -e .  # or "python setup.py develop"
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.7.0/index.html

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('Google Drive connected.')

In [ ]:
#@title copy and extract own dataset
!cp "/content/drive/MyDrive/classification.7z" "/content/classification.7z"
%cd /content/
!7z x /content/classification.7z

# Train

In [ ]:
#@title [dataset creation](https://github.com/bentrevett/pytorch-image-classification/blob/master/5_resnet.ipynb) (Split dataset in ```train``` and ```test```.)
TRAIN_RATIO = 0.9 #@param {type:"number"}
import os
import shutil
#data_dir = os.path.join(ROOT, 'CUB_200_2011')
data_dir = '/content/images/' #@param {type:"string"}
images_dir = os.path.join(data_dir, 'images')
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir) 
if os.path.exists(test_dir):
    shutil.rmtree(test_dir)
    
os.makedirs(train_dir)
os.makedirs(test_dir)

classes = os.listdir(images_dir)

for c in classes:
    
    class_dir = os.path.join(images_dir, c)
    
    images = os.listdir(class_dir)
       
    n_train = int(len(images) * TRAIN_RATIO)
    
    train_images = images[:n_train]
    test_images = images[n_train:]
    
    os.makedirs(os.path.join(train_dir, c), exist_ok = True)
    os.makedirs(os.path.join(test_dir, c), exist_ok = True)
    
    for image in train_images:
        image_src = os.path.join(class_dir, image)
        image_dst = os.path.join(train_dir, c, image) 
        shutil.copyfile(image_src, image_dst)
        
    for image in test_images:
        image_src = os.path.join(class_dir, image)
        image_dst = os.path.join(test_dir, c, image) 
        shutil.copyfile(image_src, image_dst)


It is needed to create annotation files. Warning: The classes are represented with numbers. Do that once for validation and training data. ```classes.txt``` will show the mapping.

In [ ]:
#@title Generate [needed annotation files](https://github.com/open-mmlab/mmclassification/blob/master/docs/tutorials/new_dataset.md).
import os
import glob
data_dir = '/content/images/' #@param {type:"string"}
annotation_output = '/content/train.txt' #@param {type:"string"}
class_output = '/content/classes.txt' #@param {type:"string"}
counter = 0

if os.path.exists(annotation_output):
  os.remove(annotation_output)
if os.path.exists(class_output):
  os.remove(class_output)

for subdir, dirs, files in os.walk(data_dir):
  for dir in dirs:
    folder_path = os.path.join(data_dir, dir)

    files = glob.glob(folder_path + '/**/*.png', recursive=True)
    files_jpg = glob.glob(folder_path + '/**/*.jpg', recursive=True)
    files.extend(files_jpg)

    f=open(annotation_output,'a')
    for ele in files:
        f.write(ele+" "+str(counter)+'\n')

    f.close()

    f=open(class_output,'a')
    f.write(str(dir)+" "+str(counter)+'\n')

    f.close()

    counter += 1

Example with [mmclassification/configs/resnext/resnext50_32x4d_b32x8_imagenet.py](https://github.com/open-mmlab/mmclassification/blob/master/configs/resnext/resnext50_32x4d_b32x8_imagenet.py).
```
_base_ = [
    '../_base_/models/resnext50_32x4d.py',
    '../_base_/datasets/imagenet_bs32.py',
    '../_base_/schedules/imagenet_bs256.py', '../_base_/default_runtime.py'
]
```

In [ ]:
#@title imagenet_bs32.py (edit paths)
%%writefile /content/mmclassification/configs/_base_/datasets/imagenet_bs32.py
# dataset settings
dataset_type = 'ImageNet'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', size=224),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(256, -1)),
    dict(type='CenterCrop', crop_size=224),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]
data = dict(
    samples_per_gpu=32,
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        data_prefix='/content/images',
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        data_prefix='/content/images',
        ann_file='/content/test.txt',
        pipeline=test_pipeline),
    test=dict(
        # replace `data/val` with `data/test` for standard test
        type=dataset_type,
        data_prefix='data/imagenet/val',
        ann_file='/content/test.txt',
        pipeline=test_pipeline))
evaluation = dict(interval=1, metric='accuracy')


In [ ]:
#@title train (resnext50_32x4d_b32x8_imagenet.py)
%cd /content/mmclassification/
!python tools/train.py /content/mmclassification/configs/resnext/resnext50_32x4d_b32x8_imagenet.py

# Test

In [ ]:
#@title image_demo.py (printing result instead of plotting)
%%writefile /content/mmclassification/demo/image_demo.py
from argparse import ArgumentParser

from mmcls.apis import inference_model, init_model, show_result_pyplot
import cv2

def main():
    parser = ArgumentParser()
    parser.add_argument('img', help='Image file')
    parser.add_argument('config', help='Config file')
    parser.add_argument('checkpoint', help='Checkpoint file')
    parser.add_argument(
        '--device', default='cuda:0', help='Device used for inference')
    args = parser.parse_args()

    # build the model from a config file and a checkpoint file
    model = init_model(args.config, args.checkpoint, device=args.device)
    # test a single image
    result = inference_model(model, args.img)
    # show the results
    #show_result_pyplot(model, args.img, result)
    print("result")
    print(result)


if __name__ == '__main__':
    main()


In [ ]:
#@title imagenet.py (edit classes if you want to print that)
%%writefile /content/mmclassification/mmcls/datasets/imagenet.py
import os

import numpy as np

from .base_dataset import BaseDataset
from .builder import DATASETS


def has_file_allowed_extension(filename, extensions):
    """Checks if a file is an allowed extension.

    Args:
        filename (string): path to a file

    Returns:
        bool: True if the filename ends with a known image extension
    """
    filename_lower = filename.lower()
    return any(filename_lower.endswith(ext) for ext in extensions)


def find_folders(root):
    """Find classes by folders under a root.

    Args:
        root (string): root directory of folders

    Returns:
        folder_to_idx (dict): the map from folder name to class idx
    """
    folders = [
        d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))
    ]
    folders.sort()
    folder_to_idx = {folders[i]: i for i in range(len(folders))}
    return folder_to_idx


def get_samples(root, folder_to_idx, extensions):
    """Make dataset by walking all images under a root.

    Args:
        root (string): root directory of folders
        folder_to_idx (dict): the map from class name to class idx
        extensions (tuple): allowed extensions

    Returns:
        samples (list): a list of tuple where each element is (image, label)
    """
    samples = []
    root = os.path.expanduser(root)
    for folder_name in sorted(os.listdir(root)):
        _dir = os.path.join(root, folder_name)
        if not os.path.isdir(_dir):
            continue

        for _, _, fns in sorted(os.walk(_dir)):
            for fn in sorted(fns):
                if has_file_allowed_extension(fn, extensions):
                    path = os.path.join(folder_name, fn)
                    item = (path, folder_to_idx[folder_name])
                    samples.append(item)
    return samples


@DATASETS.register_module()
class ImageNet(BaseDataset):
    """`ImageNet <http://www.image-net.org>`_ Dataset.

    This implementation is modified from
    https://github.com/pytorch/vision/blob/master/torchvision/datasets/imagenet.py  # noqa: E501
    """

    IMG_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif')
    CLASSES = [
        'tench, Tinca tinca',
        'goldfish, Carassius auratus',
        'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',  # noqa: E501
        'tiger shark, Galeocerdo cuvieri',
        'hammerhead, hammerhead shark',
        'electric ray, crampfish, numbfish, torpedo',
        'stingray',
        'cock',
        'hen',
        'ostrich, Struthio camelus',
        'brambling, Fringilla montifringilla',
        'goldfinch, Carduelis carduelis',
        'house finch, linnet, Carpodacus mexicanus',
        'junco, snowbird',
        'indigo bunting, indigo finch, indigo bird, Passerina cyanea',
        'robin, American robin, Turdus migratorius',
        'bulbul',
        'jay',
        'magpie',
        'chickadee',
        'water ouzel, dipper',
        'kite',
        'bald eagle, American eagle, Haliaeetus leucocephalus',
        'vulture',
        'great grey owl, great gray owl, Strix nebulosa',
        'European fire salamander, Salamandra salamandra',
        'common newt, Triturus vulgaris',
        'eft',
        'spotted salamander, Ambystoma maculatum',
        'axolotl, mud puppy, Ambystoma mexicanum',
        'bullfrog, Rana catesbeiana',
        'tree frog, tree-frog',
        'tailed frog, bell toad, ribbed toad, tailed toad, Ascaphus trui',
        'loggerhead, loggerhead turtle, Caretta caretta',
        'leatherback turtle, leatherback, leathery turtle, Dermochelys coriacea',  # noqa: E501
        'mud turtle',
        'terrapin',
        'box turtle, box tortoise',
        'banded gecko',
        'common iguana, iguana, Iguana iguana',
        'American chameleon, anole, Anolis carolinensis',
        'whiptail, whiptail lizard',
        'agama',
        'frilled lizard, Chlamydosaurus kingi',
        'alligator lizard',
        'Gila monster, Heloderma suspectum',
        'green lizard, Lacerta viridis',
        'African chameleon, Chamaeleo chamaeleon',
        'Komodo dragon, Komodo lizard, dragon lizard, giant lizard, Varanus komodoensis',  # noqa: E501
        'African crocodile, Nile crocodile, Crocodylus niloticus',
        'American alligator, Alligator mississipiensis',
        'triceratops',
        'thunder snake, worm snake, Carphophis amoenus',
        'ringneck snake, ring-necked snake, ring snake',
        'hognose snake, puff adder, sand viper',
        'green snake, grass snake',
        'king snake, kingsnake',
        'garter snake, grass snake',
        'water snake',
        'vine snake',
        'night snake, Hypsiglena torquata',
        'boa constrictor, Constrictor constrictor',
        'rock python, rock snake, Python sebae',
        'Indian cobra, Naja naja',
        'green mamba',
        'sea snake',
        'horned viper, cerastes, sand viper, horned asp, Cerastes cornutus',
        'diamondback, diamondback rattlesnake, Crotalus adamanteus',
        'sidewinder, horned rattlesnake, Crotalus cerastes',
        'trilobite',
        'harvestman, daddy longlegs, Phalangium opilio',
        'scorpion',
        'black and gold garden spider, Argiope aurantia',
        'barn spider, Araneus cavaticus',
        'garden spider, Aranea diademata',
        'black widow, Latrodectus mactans',
        'tarantula',
        'wolf spider, hunting spider',
        'tick',
        'centipede',
        'black grouse',
        'ptarmigan',
        'ruffed grouse, partridge, Bonasa umbellus',
        'prairie chicken, prairie grouse, prairie fowl',
        'peacock',
        'quail',
        'partridge',
        'African grey, African gray, Psittacus erithacus',
        'macaw',
        'sulphur-crested cockatoo, Kakatoe galerita, Cacatua galerita',
        'lorikeet',
        'coucal',
        'bee eater',
        'hornbill',
        'hummingbird',
        'jacamar',
        'toucan',
        'drake',
        'red-breasted merganser, Mergus serrator',
        'goose',
        'black swan, Cygnus atratus',
        'tusker',
        'echidna, spiny anteater, anteater',
        'platypus, duckbill, duckbilled platypus, duck-billed platypus, Ornithorhynchus anatinus',  # noqa: E501
        'wallaby, brush kangaroo',
        'koala, koala bear, kangaroo bear, native bear, Phascolarctos cinereus',  # noqa: E501
        'wombat',
        'jellyfish',
        'sea anemone, anemone',
        'brain coral',
        'flatworm, platyhelminth',
        'nematode, nematode worm, roundworm',
        'conch',
        'snail',
        'slug',
        'sea slug, nudibranch',
        'chiton, coat-of-mail shell, sea cradle, polyplacophore',
        'chambered nautilus, pearly nautilus, nautilus',
        'Dungeness crab, Cancer magister',
        'rock crab, Cancer irroratus',
        'fiddler crab',
        'king crab, Alaska crab, Alaskan king crab, Alaska king crab, Paralithodes camtschatica',  # noqa: E501
        'American lobster, Northern lobster, Maine lobster, Homarus americanus',  # noqa: E501
        'spiny lobster, langouste, rock lobster, crawfish, crayfish, sea crawfish',  # noqa: E501
        'crayfish, crawfish, crawdad, crawdaddy',
        'hermit crab',
        'isopod',
        'white stork, Ciconia ciconia',
        'black stork, Ciconia nigra',
        'spoonbill',
        'flamingo',
        'little blue heron, Egretta caerulea',
        'American egret, great white heron, Egretta albus',
        'bittern',
        'crane',
        'limpkin, Aramus pictus',
        'European gallinule, Porphyrio porphyrio',
        'American coot, marsh hen, mud hen, water hen, Fulica americana',
        'bustard',
        'ruddy turnstone, Arenaria interpres',
        'red-backed sandpiper, dunlin, Erolia alpina',
        'redshank, Tringa totanus',
        'dowitcher',
        'oystercatcher, oyster catcher',
        'pelican',
        'king penguin, Aptenodytes patagonica',
        'albatross, mollymawk',
        'grey whale, gray whale, devilfish, Eschrichtius gibbosus, Eschrichtius robustus',  # noqa: E501
        'killer whale, killer, orca, grampus, sea wolf, Orcinus orca',
        'dugong, Dugong dugon',
        'sea lion',
        'Chihuahua',
        'Japanese spaniel',
        'Maltese dog, Maltese terrier, Maltese',
        'Pekinese, Pekingese, Peke',
        'Shih-Tzu',
        'Blenheim spaniel',
        'papillon',
        'toy terrier',
        'Rhodesian ridgeback',
        'Afghan hound, Afghan',
        'basset, basset hound',
        'beagle',
        'bloodhound, sleuthhound',
        'bluetick',
        'black-and-tan coonhound',
        'Walker hound, Walker foxhound',
        'English foxhound',
        'redbone',
        'borzoi, Russian wolfhound',
        'Irish wolfhound',
        'Italian greyhound',
        'whippet',
        'Ibizan hound, Ibizan Podenco',
        'Norwegian elkhound, elkhound',
        'otterhound, otter hound',
        'Saluki, gazelle hound',
        'Scottish deerhound, deerhound',
        'Weimaraner',
        'Staffordshire bullterrier, Staffordshire bull terrier',
        'American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier',  # noqa: E501
        'Bedlington terrier',
        'Border terrier',
        'Kerry blue terrier',
        'Irish terrier',
        'Norfolk terrier',
        'Norwich terrier',
        'Yorkshire terrier',
        'wire-haired fox terrier',
        'Lakeland terrier',
        'Sealyham terrier, Sealyham',
        'Airedale, Airedale terrier',
        'cairn, cairn terrier',
        'Australian terrier',
        'Dandie Dinmont, Dandie Dinmont terrier',
        'Boston bull, Boston terrier',
        'miniature schnauzer',
        'giant schnauzer',
        'standard schnauzer',
        'Scotch terrier, Scottish terrier, Scottie',
        'Tibetan terrier, chrysanthemum dog',
        'silky terrier, Sydney silky',
        'soft-coated wheaten terrier',
        'West Highland white terrier',
        'Lhasa, Lhasa apso',
        'flat-coated retriever',
        'curly-coated retriever',
        'golden retriever',
        'Labrador retriever',
        'Chesapeake Bay retriever',
        'German short-haired pointer',
        'vizsla, Hungarian pointer',
        'English setter',
        'Irish setter, red setter',
        'Gordon setter',
        'Brittany spaniel',
        'clumber, clumber spaniel',
        'English springer, English springer spaniel',
        'Welsh springer spaniel',
        'cocker spaniel, English cocker spaniel, cocker',
        'Sussex spaniel',
        'Irish water spaniel',
        'kuvasz',
        'schipperke',
        'groenendael',
        'malinois',
        'briard',
        'kelpie',
        'komondor',
        'Old English sheepdog, bobtail',
        'Shetland sheepdog, Shetland sheep dog, Shetland',
        'collie',
        'Border collie',
        'Bouvier des Flandres, Bouviers des Flandres',
        'Rottweiler',
        'German shepherd, German shepherd dog, German police dog, alsatian',
        'Doberman, Doberman pinscher',
        'miniature pinscher',
        'Greater Swiss Mountain dog',
        'Bernese mountain dog',
        'Appenzeller',
        'EntleBucher',
        'boxer',
        'bull mastiff',
        'Tibetan mastiff',
        'French bulldog',
        'Great Dane',
        'Saint Bernard, St Bernard',
        'Eskimo dog, husky',
        'malamute, malemute, Alaskan malamute',
        'Siberian husky',
        'dalmatian, coach dog, carriage dog',
        'affenpinscher, monkey pinscher, monkey dog',
        'basenji',
        'pug, pug-dog',
        'Leonberg',
        'Newfoundland, Newfoundland dog',
        'Great Pyrenees',
        'Samoyed, Samoyede',
        'Pomeranian',
        'chow, chow chow',
        'keeshond',
        'Brabancon griffon',
        'Pembroke, Pembroke Welsh corgi',
        'Cardigan, Cardigan Welsh corgi',
        'toy poodle',
        'miniature poodle',
        'standard poodle',
        'Mexican hairless',
        'timber wolf, grey wolf, gray wolf, Canis lupus',
        'white wolf, Arctic wolf, Canis lupus tundrarum',
        'red wolf, maned wolf, Canis rufus, Canis niger',
        'coyote, prairie wolf, brush wolf, Canis latrans',
        'dingo, warrigal, warragal, Canis dingo',
        'dhole, Cuon alpinus',
        'African hunting dog, hyena dog, Cape hunting dog, Lycaon pictus',
        'hyena, hyaena',
        'red fox, Vulpes vulpes',
        'kit fox, Vulpes macrotis',
        'Arctic fox, white fox, Alopex lagopus',
        'grey fox, gray fox, Urocyon cinereoargenteus',
        'tabby, tabby cat',
        'tiger cat',
        'Persian cat',
        'Siamese cat, Siamese',
        'Egyptian cat',
        'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor',  # noqa: E501
        'lynx, catamount',
        'leopard, Panthera pardus',
        'snow leopard, ounce, Panthera uncia',
        'jaguar, panther, Panthera onca, Felis onca',
        'lion, king of beasts, Panthera leo',
        'tiger, Panthera tigris',
        'cheetah, chetah, Acinonyx jubatus',
        'brown bear, bruin, Ursus arctos',
        'American black bear, black bear, Ursus americanus, Euarctos americanus',  # noqa: E501
        'ice bear, polar bear, Ursus Maritimus, Thalarctos maritimus',
        'sloth bear, Melursus ursinus, Ursus ursinus',
        'mongoose',
        'meerkat, mierkat',
        'tiger beetle',
        'ladybug, ladybeetle, lady beetle, ladybird, ladybird beetle',
        'ground beetle, carabid beetle',
        'long-horned beetle, longicorn, longicorn beetle',
        'leaf beetle, chrysomelid',
        'dung beetle',
        'rhinoceros beetle',
        'weevil',
        'fly',
        'bee',
        'ant, emmet, pismire',
        'grasshopper, hopper',
        'cricket',
        'walking stick, walkingstick, stick insect',
        'cockroach, roach',
        'mantis, mantid',
        'cicada, cicala',
        'leafhopper',
        'lacewing, lacewing fly',
        "dragonfly, darning needle, devil's darning needle, sewing needle, snake feeder, snake doctor, mosquito hawk, skeeter hawk",  # noqa: E501
        'damselfly',
        'admiral',
        'ringlet, ringlet butterfly',
        'monarch, monarch butterfly, milkweed butterfly, Danaus plexippus',
        'cabbage butterfly',
        'sulphur butterfly, sulfur butterfly',
        'lycaenid, lycaenid butterfly',
        'starfish, sea star',
        'sea urchin',
        'sea cucumber, holothurian',
        'wood rabbit, cottontail, cottontail rabbit',
        'hare',
        'Angora, Angora rabbit',
        'hamster',
        'porcupine, hedgehog',
        'fox squirrel, eastern fox squirrel, Sciurus niger',
        'marmot',
        'beaver',
        'guinea pig, Cavia cobaya',
        'sorrel',
        'zebra',
        'hog, pig, grunter, squealer, Sus scrofa',
        'wild boar, boar, Sus scrofa',
        'warthog',
        'hippopotamus, hippo, river horse, Hippopotamus amphibius',
        'ox',
        'water buffalo, water ox, Asiatic buffalo, Bubalus bubalis',
        'bison',
        'ram, tup',
        'bighorn, bighorn sheep, cimarron, Rocky Mountain bighorn, Rocky Mountain sheep, Ovis canadensis',  # noqa: E501
        'ibex, Capra ibex',
        'hartebeest',
        'impala, Aepyceros melampus',
        'gazelle',
        'Arabian camel, dromedary, Camelus dromedarius',
        'llama',
        'weasel',
        'mink',
        'polecat, fitch, foulmart, foumart, Mustela putorius',
        'black-footed ferret, ferret, Mustela nigripes',
        'otter',
        'skunk, polecat, wood pussy',
        'badger',
        'armadillo',
        'three-toed sloth, ai, Bradypus tridactylus',
        'orangutan, orang, orangutang, Pongo pygmaeus',
        'gorilla, Gorilla gorilla',
        'chimpanzee, chimp, Pan troglodytes',
        'gibbon, Hylobates lar',
        'siamang, Hylobates syndactylus, Symphalangus syndactylus',
        'guenon, guenon monkey',
        'patas, hussar monkey, Erythrocebus patas',
        'baboon',
        'macaque',
        'langur',
        'colobus, colobus monkey',
        'proboscis monkey, Nasalis larvatus',
        'marmoset',
        'capuchin, ringtail, Cebus capucinus',
        'howler monkey, howler',
        'titi, titi monkey',
        'spider monkey, Ateles geoffroyi',
        'squirrel monkey, Saimiri sciureus',
        'Madagascar cat, ring-tailed lemur, Lemur catta',
        'indri, indris, Indri indri, Indri brevicaudatus',
        'Indian elephant, Elephas maximus',
        'African elephant, Loxodonta africana',
        'lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens',
        'giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca',
        'barracouta, snoek',
        'eel',
        'coho, cohoe, coho salmon, blue jack, silver salmon, Oncorhynchus kisutch',  # noqa: E501
        'rock beauty, Holocanthus tricolor',
        'anemone fish',
        'sturgeon',
        'gar, garfish, garpike, billfish, Lepisosteus osseus',
        'lionfish',
        'puffer, pufferfish, blowfish, globefish',
        'abacus',
        'abaya',
        "academic gown, academic robe, judge's robe",
        'accordion, piano accordion, squeeze box',
        'acoustic guitar',
        'aircraft carrier, carrier, flattop, attack aircraft carrier',
        'airliner',
        'airship, dirigible',
        'altar',
        'ambulance',
        'amphibian, amphibious vehicle',
        'analog clock',
        'apiary, bee house',
        'apron',
        'ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, dustbin, trash barrel, trash bin',  # noqa: E501
        'assault rifle, assault gun',
        'backpack, back pack, knapsack, packsack, rucksack, haversack',
        'bakery, bakeshop, bakehouse',
        'balance beam, beam',
        'balloon',
        'ballpoint, ballpoint pen, ballpen, Biro',
        'Band Aid',
        'banjo',
        'bannister, banister, balustrade, balusters, handrail',
        'barbell',
        'barber chair',
        'barbershop',
        'barn',
        'barometer',
        'barrel, cask',
        'barrow, garden cart, lawn cart, wheelbarrow',
        'baseball',
        'basketball',
        'bassinet',
        'bassoon',
        'bathing cap, swimming cap',
        'bath towel',
        'bathtub, bathing tub, bath, tub',
        'beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon',  # noqa: E501
        'beacon, lighthouse, beacon light, pharos',
        'beaker',
        'bearskin, busby, shako',
        'beer bottle',
        'beer glass',
        'bell cote, bell cot',
        'bib',
        'bicycle-built-for-two, tandem bicycle, tandem',
        'bikini, two-piece',
        'binder, ring-binder',
        'binoculars, field glasses, opera glasses',
        'birdhouse',
        'boathouse',
        'bobsled, bobsleigh, bob',
        'bolo tie, bolo, bola tie, bola',
        'bonnet, poke bonnet',
        'bookcase',
        'bookshop, bookstore, bookstall',
        'bottlecap',
        'bow',
        'bow tie, bow-tie, bowtie',
        'brass, memorial tablet, plaque',
        'brassiere, bra, bandeau',
        'breakwater, groin, groyne, mole, bulwark, seawall, jetty',
        'breastplate, aegis, egis',
        'broom',
        'bucket, pail',
        'buckle',
        'bulletproof vest',
        'bullet train, bullet',
        'butcher shop, meat market',
        'cab, hack, taxi, taxicab',
        'caldron, cauldron',
        'candle, taper, wax light',
        'cannon',
        'canoe',
        'can opener, tin opener',
        'cardigan',
        'car mirror',
        'carousel, carrousel, merry-go-round, roundabout, whirligig',
        "carpenter's kit, tool kit",
        'carton',
        'car wheel',
        'cash machine, cash dispenser, automated teller machine, automatic teller machine, automated teller, automatic teller, ATM',  # noqa: E501
        'cassette',
        'cassette player',
        'castle',
        'catamaran',
        'CD player',
        'cello, violoncello',
        'cellular telephone, cellular phone, cellphone, cell, mobile phone',
        'chain',
        'chainlink fence',
        'chain mail, ring mail, mail, chain armor, chain armour, ring armor, ring armour',  # noqa: E501
        'chain saw, chainsaw',
        'chest',
        'chiffonier, commode',
        'chime, bell, gong',
        'china cabinet, china closet',
        'Christmas stocking',
        'church, church building',
        'cinema, movie theater, movie theatre, movie house, picture palace',
        'cleaver, meat cleaver, chopper',
        'cliff dwelling',
        'cloak',
        'clog, geta, patten, sabot',
        'cocktail shaker',
        'coffee mug',
        'coffeepot',
        'coil, spiral, volute, whorl, helix',
        'combination lock',
        'computer keyboard, keypad',
        'confectionery, confectionary, candy store',
        'container ship, containership, container vessel',
        'convertible',
        'corkscrew, bottle screw',
        'cornet, horn, trumpet, trump',
        'cowboy boot',
        'cowboy hat, ten-gallon hat',
        'cradle',
        'crane',
        'crash helmet',
        'crate',
        'crib, cot',
        'Crock Pot',
        'croquet ball',
        'crutch',
        'cuirass',
        'dam, dike, dyke',
        'desk',
        'desktop computer',
        'dial telephone, dial phone',
        'diaper, nappy, napkin',
        'digital clock',
        'digital watch',
        'dining table, board',
        'dishrag, dishcloth',
        'dishwasher, dish washer, dishwashing machine',
        'disk brake, disc brake',
        'dock, dockage, docking facility',
        'dogsled, dog sled, dog sleigh',
        'dome',
        'doormat, welcome mat',
        'drilling platform, offshore rig',
        'drum, membranophone, tympan',
        'drumstick',
        'dumbbell',
        'Dutch oven',
        'electric fan, blower',
        'electric guitar',
        'electric locomotive',
        'entertainment center',
        'envelope',
        'espresso maker',
        'face powder',
        'feather boa, boa',
        'file, file cabinet, filing cabinet',
        'fireboat',
        'fire engine, fire truck',
        'fire screen, fireguard',
        'flagpole, flagstaff',
        'flute, transverse flute',
        'folding chair',
        'football helmet',
        'forklift',
        'fountain',
        'fountain pen',
        'four-poster',
        'freight car',
        'French horn, horn',
        'frying pan, frypan, skillet',
        'fur coat',
        'garbage truck, dustcart',
        'gasmask, respirator, gas helmet',
        'gas pump, gasoline pump, petrol pump, island dispenser',
        'goblet',
        'go-kart',
        'golf ball',
        'golfcart, golf cart',
        'gondola',
        'gong, tam-tam',
        'gown',
        'grand piano, grand',
        'greenhouse, nursery, glasshouse',
        'grille, radiator grille',
        'grocery store, grocery, food market, market',
        'guillotine',
        'hair slide',
        'hair spray',
        'half track',
        'hammer',
        'hamper',
        'hand blower, blow dryer, blow drier, hair dryer, hair drier',
        'hand-held computer, hand-held microcomputer',
        'handkerchief, hankie, hanky, hankey',
        'hard disc, hard disk, fixed disk',
        'harmonica, mouth organ, harp, mouth harp',
        'harp',
        'harvester, reaper',
        'hatchet',
        'holster',
        'home theater, home theatre',
        'honeycomb',
        'hook, claw',
        'hoopskirt, crinoline',
        'horizontal bar, high bar',
        'horse cart, horse-cart',
        'hourglass',
        'iPod',
        'iron, smoothing iron',
        "jack-o'-lantern",
        'jean, blue jean, denim',
        'jeep, landrover',
        'jersey, T-shirt, tee shirt',
        'jigsaw puzzle',
        'jinrikisha, ricksha, rickshaw',
        'joystick',
        'kimono',
        'knee pad',
        'knot',
        'lab coat, laboratory coat',
        'ladle',
        'lampshade, lamp shade',
        'laptop, laptop computer',
        'lawn mower, mower',
        'lens cap, lens cover',
        'letter opener, paper knife, paperknife',
        'library',
        'lifeboat',
        'lighter, light, igniter, ignitor',
        'limousine, limo',
        'liner, ocean liner',
        'lipstick, lip rouge',
        'Loafer',
        'lotion',
        'loudspeaker, speaker, speaker unit, loudspeaker system, speaker system',  # noqa: E501
        "loupe, jeweler's loupe",
        'lumbermill, sawmill',
        'magnetic compass',
        'mailbag, postbag',
        'mailbox, letter box',
        'maillot',
        'maillot, tank suit',
        'manhole cover',
        'maraca',
        'marimba, xylophone',
        'mask',
        'matchstick',
        'maypole',
        'maze, labyrinth',
        'measuring cup',
        'medicine chest, medicine cabinet',
        'megalith, megalithic structure',
        'microphone, mike',
        'microwave, microwave oven',
        'military uniform',
        'milk can',
        'minibus',
        'miniskirt, mini',
        'minivan',
        'missile',
        'mitten',
        'mixing bowl',
        'mobile home, manufactured home',
        'Model T',
        'modem',
        'monastery',
        'monitor',
        'moped',
        'mortar',
        'mortarboard',
        'mosque',
        'mosquito net',
        'motor scooter, scooter',
        'mountain bike, all-terrain bike, off-roader',
        'mountain tent',
        'mouse, computer mouse',
        'mousetrap',
        'moving van',
        'muzzle',
        'nail',
        'neck brace',
        'necklace',
        'nipple',
        'notebook, notebook computer',
        'obelisk',
        'oboe, hautboy, hautbois',
        'ocarina, sweet potato',
        'odometer, hodometer, mileometer, milometer',
        'oil filter',
        'organ, pipe organ',
        'oscilloscope, scope, cathode-ray oscilloscope, CRO',
        'overskirt',
        'oxcart',
        'oxygen mask',
        'packet',
        'paddle, boat paddle',
        'paddlewheel, paddle wheel',
        'padlock',
        'paintbrush',
        "pajama, pyjama, pj's, jammies",
        'palace',
        'panpipe, pandean pipe, syrinx',
        'paper towel',
        'parachute, chute',
        'parallel bars, bars',
        'park bench',
        'parking meter',
        'passenger car, coach, carriage',
        'patio, terrace',
        'pay-phone, pay-station',
        'pedestal, plinth, footstall',
        'pencil box, pencil case',
        'pencil sharpener',
        'perfume, essence',
        'Petri dish',
        'photocopier',
        'pick, plectrum, plectron',
        'pickelhaube',
        'picket fence, paling',
        'pickup, pickup truck',
        'pier',
        'piggy bank, penny bank',
        'pill bottle',
        'pillow',
        'ping-pong ball',
        'pinwheel',
        'pirate, pirate ship',
        'pitcher, ewer',
        "plane, carpenter's plane, woodworking plane",
        'planetarium',
        'plastic bag',
        'plate rack',
        'plow, plough',
        "plunger, plumber's helper",
        'Polaroid camera, Polaroid Land camera',
        'pole',
        'police van, police wagon, paddy wagon, patrol wagon, wagon, black Maria',  # noqa: E501
        'poncho',
        'pool table, billiard table, snooker table',
        'pop bottle, soda bottle',
        'pot, flowerpot',
        "potter's wheel",
        'power drill',
        'prayer rug, prayer mat',
        'printer',
        'prison, prison house',
        'projectile, missile',
        'projector',
        'puck, hockey puck',
        'punching bag, punch bag, punching ball, punchball',
        'purse',
        'quill, quill pen',
        'quilt, comforter, comfort, puff',
        'racer, race car, racing car',
        'racket, racquet',
        'radiator',
        'radio, wireless',
        'radio telescope, radio reflector',
        'rain barrel',
        'recreational vehicle, RV, R.V.',
        'reel',
        'reflex camera',
        'refrigerator, icebox',
        'remote control, remote',
        'restaurant, eating house, eating place, eatery',
        'revolver, six-gun, six-shooter',
        'rifle',
        'rocking chair, rocker',
        'rotisserie',
        'rubber eraser, rubber, pencil eraser',
        'rugby ball',
        'rule, ruler',
        'running shoe',
        'safe',
        'safety pin',
        'saltshaker, salt shaker',
        'sandal',
        'sarong',
        'sax, saxophone',
        'scabbard',
        'scale, weighing machine',
        'school bus',
        'schooner',
        'scoreboard',
        'screen, CRT screen',
        'screw',
        'screwdriver',
        'seat belt, seatbelt',
        'sewing machine',
        'shield, buckler',
        'shoe shop, shoe-shop, shoe store',
        'shoji',
        'shopping basket',
        'shopping cart',
        'shovel',
        'shower cap',
        'shower curtain',
        'ski',
        'ski mask',
        'sleeping bag',
        'slide rule, slipstick',
        'sliding door',
        'slot, one-armed bandit',
        'snorkel',
        'snowmobile',
        'snowplow, snowplough',
        'soap dispenser',
        'soccer ball',
        'sock',
        'solar dish, solar collector, solar furnace',
        'sombrero',
        'soup bowl',
        'space bar',
        'space heater',
        'space shuttle',
        'spatula',
        'speedboat',
        "spider web, spider's web",
        'spindle',
        'sports car, sport car',
        'spotlight, spot',
        'stage',
        'steam locomotive',
        'steel arch bridge',
        'steel drum',
        'stethoscope',
        'stole',
        'stone wall',
        'stopwatch, stop watch',
        'stove',
        'strainer',
        'streetcar, tram, tramcar, trolley, trolley car',
        'stretcher',
        'studio couch, day bed',
        'stupa, tope',
        'submarine, pigboat, sub, U-boat',
        'suit, suit of clothes',
        'sundial',
        'sunglass',
        'sunglasses, dark glasses, shades',
        'sunscreen, sunblock, sun blocker',
        'suspension bridge',
        'swab, swob, mop',
        'sweatshirt',
        'swimming trunks, bathing trunks',
        'swing',
        'switch, electric switch, electrical switch',
        'syringe',
        'table lamp',
        'tank, army tank, armored combat vehicle, armoured combat vehicle',
        'tape player',
        'teapot',
        'teddy, teddy bear',
        'television, television system',
        'tennis ball',
        'thatch, thatched roof',
        'theater curtain, theatre curtain',
        'thimble',
        'thresher, thrasher, threshing machine',
        'throne',
        'tile roof',
        'toaster',
        'tobacco shop, tobacconist shop, tobacconist',
        'toilet seat',
        'torch',
        'totem pole',
        'tow truck, tow car, wrecker',
        'toyshop',
        'tractor',
        'trailer truck, tractor trailer, trucking rig, rig, articulated lorry, semi',  # noqa: E501
        'tray',
        'trench coat',
        'tricycle, trike, velocipede',
        'trimaran',
        'tripod',
        'triumphal arch',
        'trolleybus, trolley coach, trackless trolley',
        'trombone',
        'tub, vat',
        'turnstile',
        'typewriter keyboard',
        'umbrella',
        'unicycle, monocycle',
        'upright, upright piano',
        'vacuum, vacuum cleaner',
        'vase',
        'vault',
        'velvet',
        'vending machine',
        'vestment',
        'viaduct',
        'violin, fiddle',
        'volleyball',
        'waffle iron',
        'wall clock',
        'wallet, billfold, notecase, pocketbook',
        'wardrobe, closet, press',
        'warplane, military plane',
        'washbasin, handbasin, washbowl, lavabo, wash-hand basin',
        'washer, automatic washer, washing machine',
        'water bottle',
        'water jug',
        'water tower',
        'whiskey jug',
        'whistle',
        'wig',
        'window screen',
        'window shade',
        'Windsor tie',
        'wine bottle',
        'wing',
        'wok',
        'wooden spoon',
        'wool, woolen, woollen',
        'worm fence, snake fence, snake-rail fence, Virginia fence',
        'wreck',
        'yawl',
        'yurt',
        'web site, website, internet site, site',
        'comic book',
        'crossword puzzle, crossword',
        'street sign',
        'traffic light, traffic signal, stoplight',
        'book jacket, dust cover, dust jacket, dust wrapper',
        'menu',
        'plate',
        'guacamole',
        'consomme',
        'hot pot, hotpot',
        'trifle',
        'ice cream, icecream',
        'ice lolly, lolly, lollipop, popsicle',
        'French loaf',
        'bagel, beigel',
        'pretzel',
        'cheeseburger',
        'hotdog, hot dog, red hot',
        'mashed potato',
        'head cabbage',
        'broccoli',
        'cauliflower',
        'zucchini, courgette',
        'spaghetti squash',
        'acorn squash',
        'butternut squash',
        'cucumber, cuke',
        'artichoke, globe artichoke',
        'bell pepper',
        'cardoon',
        'mushroom',
        'Granny Smith',
        'strawberry',
        'orange',
        'lemon',
        'fig',
        'pineapple, ananas',
        'banana',
        'jackfruit, jak, jack',
        'custard apple',
        'pomegranate',
        'hay',
        'carbonara',
        'chocolate sauce, chocolate syrup',
        'dough',
        'meat loaf, meatloaf',
        'pizza, pizza pie',
        'potpie',
        'burrito',
        'red wine',
        'espresso',
        'cup',
        'eggnog',
        'alp',
        'bubble',
        'cliff, drop, drop-off',
        'coral reef',
        'geyser',
        'lakeside, lakeshore',
        'promontory, headland, head, foreland',
        'sandbar, sand bar',
        'seashore, coast, seacoast, sea-coast',
        'valley, vale',
        'volcano',
        'ballplayer, baseball player',
        'groom, bridegroom',
        'scuba diver',
        'rapeseed',
        'daisy',
        "yellow lady's slipper, yellow lady-slipper, Cypripedium calceolus, Cypripedium parviflorum",  # noqa: E501
        'corn',
        'acorn',
        'hip, rose hip, rosehip',
        'buckeye, horse chestnut, conker',
        'coral fungus',
        'agaric',
        'gyromitra',
        'stinkhorn, carrion fungus',
        'earthstar',
        'hen-of-the-woods, hen of the woods, Polyporus frondosus, Grifola frondosa',  # noqa: E501
        'bolete',
        'ear, spike, capitulum',
        'toilet tissue, toilet paper, bathroom tissue'
    ]

    def load_annotations(self):
        if self.ann_file is None:
            folder_to_idx = find_folders(self.data_prefix)
            samples = get_samples(
                self.data_prefix,
                folder_to_idx,
                extensions=self.IMG_EXTENSIONS)
            if len(samples) == 0:
                raise (RuntimeError('Found 0 files in subfolders of: '
                                    f'{self.data_prefix}. '
                                    'Supported extensions are: '
                                    f'{",".join(self.IMG_EXTENSIONS)}'))

            self.folder_to_idx = folder_to_idx
        elif isinstance(self.ann_file, str):
            with open(self.ann_file) as f:
                samples = [x.strip().split(' ') for x in f.readlines()]
        else:
            raise TypeError('ann_file must be a str or None')
        self.samples = samples

        data_infos = []
        for filename, gt_label in self.samples:
            info = {'img_prefix': self.data_prefix}
            info['img_info'] = {'filename': filename}
            info['gt_label'] = np.array(gt_label, dtype=np.int64)
            data_infos.append(info)
        return data_infos


In [ ]:
#@title print classification result
%cd /content/mmclassification
!python demo/image_demo.py /content/image.png \
    /content/mmclassification/configs/resnext/resnext50_32x4d_b32x8_imagenet.py /content/mmclassification/work_dirs/resnext50_32x4d_b32x8_imagenet/latest.pth

In [ ]:
#@title getting topk metrics
!python tools/test.py /content/mmclassification/configs/resnext/resnext50_32x4d_b32x8_imagenet.py \
    /content/mmclassification/work_dirs/resnext50_32x4d_b32x8_imagenet/latest.pth